# Superdense Algorithm
> First I load some helper functions I've created in the past to suit my needs. They work for one, two, and three bit combinations

In [1]:
three_bits = ["000","001","010","011","100","101","110","111"]
two_bits = ["00","01","10","11"]
one_bits = ["0","1"]


def measureFirstQubit(circuit_state):
    qubit_factors = circuit_state
    factors = absolute_square(qubit_factors)
    outcomes = {"0":0,"1":0}
    
    i = 0
    for value in factors.values():
        if i < 4:
            outcomes["0"] += round(value,3)
        else:
            outcomes["1"] += round(value,3)
        i += 1
    return normalization(outcomes)


def normalization(probabilities):
    N = 0

    possible_values = three_bits
    if len(probabilities) == 4:
        possible_values = two_bits
    if len(probabilities) == 2:
        possible_values = one_bits
        
    for combo in possible_values:
        N += probabilities[combo]

    if N != 1:
        for combo in possible_values:
            probabilities[combo] = probabilities[combo] / N
    return probabilities
                
def absolute_square(factors):
    absolute_squares = {}
    
    # Adjusting function based on if were calculating one bit, two bits or three bits.
    bit_combinations = three_bits
    
    if len(factors) == 2:
        bit_combinations = one_bits
    if len(factors) == 4:
        bit_combinations = two_bits
    
    # Loop through mapping probabilities to bit values
    for index in range(len(factors)):
        probability_amplitude = abs(factors[index].real) ** 2 + abs(factors[index].imag) ** 2
        absolute_squares[bit_combinations[index]] = round(probability_amplitude,3)

    return absolute_squares


> These are superdense specific helper algorithms. Note: Hadamard + Controlled Not = Entanglement

In [2]:
import dwave.gate.operations as gate
from dwave.gate.simulator import simulate
from dwave.gate import Circuit
from dwave.gate.registers.registers import ClassicalRegister, QuantumRegister
from math import pi

def entangle_qubits(qubit_one, qubit_two, circuit):
    gate.Hadamard(qubit_one)
    gate.CX(control=qubit_one, target=qubit_two)
    simulate(circuit)

def measureQubits(circuit_state, display):
    qubit_factors = circuit_state
    # circuit_state is the corresponding [alpha, beta, gamma, delta]
    # ie. state |00> = [1.+0.j 0.+0.j 0.+0.j 0.+0.j]

    if display:
        print(qubit_factors)
    
    # Calculates the probabilities via the absolute square followed by normalization
    factors = normalization(absolute_square(qubit_factors))
    
    # This calculates the probabilities of each outcome
    # ie. {'00': 1.0, '01': 0.0, '10': 0.0, '11': 0.0}
    
    # Finds which term has a 100% probability, crashes the program otherwise.
    # Thankfully, the program never crashes.
    
    for factor in factors.items():
        if factor[1] == 1:
            value = factor[0]
            if display:
                print("Bob Measures: "+factor[0])
    return value

> This function performs the superdense algorithm. "display" simply adds new print statements to the function.

In [3]:
def superdense_algorithm(messages, display):
    # Circuit has two qubits, zero classical bits
    circuit = Circuit(2,0)
 
    with circuit.context as ((alice_qubit, bob_qubit), cbits):
        # Performs a Hadamard and CNOT
        
        entangle_qubits(alice_qubit, bob_qubit, circuit)
    
        binary_message = ""
        # The messages array is all two-bit strings, ie. ["00","11","01"]
        for message in messages:
            if display:
                print("Sending "+message+" to Bob.")
            if message[0] == "1":
                if display:
                    print("Applied Gate Z to Alice's Qubit.")
                gate.Z(alice_qubit)
            if message[1] == "1":
                if display:
                    print("Applied Gate X to Alice's Qubit.")
                gate.X(alice_qubit)
         
            if display:
                print("Alice has transferred her qubit to Bob!")

            # Operations that Bob performs on Qubits.
            gate.CX(control=alice_qubit, target=bob_qubit)
            gate.Hadamard(alice_qubit)

            simulate(circuit)
         
            binary_message += measureQubits(circuit.state, display)
            
            
            # Apply Unitary Matrix to reset circuit state.
            gate.Hadamard(alice_qubit)
            gate.CX(control=alice_qubit, target=bob_qubit)

            # Undo X/Z gates
            if message[1] == "1":
                gate.X(alice_qubit)
            if message[0] == "1":
                gate.Z(alice_qubit)

            # Circuit is reset to original state
            simulate(circuit)
            
            if display:
                print()


    return binary_message


> This is an example of how to use superdense algorithm to make a practical transfer of data.

In [4]:

transaction_1 = {
    "Transaction ID": 1,
    "Time": "8am",
    "Merchant": "Amazon",
    "Amount": 129.99,
    "Currency": "CAD",
    "Card Type": "Visa",
    "Cardholder Name": "John Doe"
}

transaction_2 = {
    "Transaction ID": 2,
    "Time": "10pm",
    "Merchant": "Starbucks",
    "Amount": 4.75,
    "Currency": "USD",
    "Card Type": "Mastercard",
    "Cardholder Name": "Jane Smith"
}

transaction_3 = {
    "Transaction ID": 3,
    "Time": "12am",
    "Merchant": "Walmart",
    "Amount": 87.50,
    "Currency": "USD",
    "Card Type": "Visa",
    "Cardholder Name": "Robert Johnson"
}

transaction_4 = {
    "Transaction ID": 4,
    "Time": "2am",
    "Merchant": "Target",
    "Amount": 67.30,
    "Currency": "CAD",
    "Card Type": "Amex",
    "Cardholder Name": "Emily Brown"
}

transaction_5 = {
    "Transaction ID": 5,
    "Time": "3am",
    "Merchant": "Gas Station",
    "Amount": 451.20,
    "Currency": "EUR",
    "Card Type": "Discover",
    "Cardholder Name": "Michael Davis"
}
transactions = [transaction_1, transaction_2, transaction_3, transaction_4, transaction_5]


def encode_binary(transaction_data):
    binary_data = {}

    # Convert Transaction ID to binary
    binary_data["Transaction ID"] = bin(transaction_data["Transaction ID"])[2:].zfill(4)

    # Convert Time to binary (since it's a string, we won't convert it)
    binary_data["Time"] = ''.join(format(ord(char), '08b') for char in transaction_data["Time"])

    # Convert Merchant to binary
    binary_data["Merchant"] = ''.join(format(ord(char), '08b') for char in transaction_data["Merchant"])

    # Convert Amount to binary
    binary_data["Amount"] = bin(int(transaction_data["Amount"] * 100))[2:].zfill(16)  # Convert to cents and then to binary

    # Convert Currency to binary
    binary_data["Currency"] = ''.join(format(ord(char), '08b') for char in transaction_data["Currency"])

    # Convert Card Type to binary
    binary_data["Card Type"] = ''.join(format(ord(char), '08b') for char in transaction_data["Card Type"])

    # Convert Cardholder Name to binary
    binary_data["Cardholder Name"] = ''.join(format(ord(char), '08b') for char in transaction_data["Cardholder Name"])

    return binary_data

def decode_binary(binary_data):
    transaction_data = {}

    # Convert binary Transaction ID back to integer
    transaction_data["Transaction ID"] = int(binary_data["Transaction ID"], 2)

    # Convert binary Time back to string
    time_binary = binary_data["Time"]
    time_string = ''.join(chr(int(time_binary[i:i+8], 2)) for i in range(0, len(time_binary), 8))
    transaction_data["Time"] = time_string

    # Convert binary Merchant back to string
    transaction_data["Merchant"] = ''.join(chr(int(binary_data["Merchant"][i:i+8], 2)) for i in range(0, len(binary_data["Merchant"]), 8))

    # Convert binary Amount back to float
    transaction_data["Amount"] = int(binary_data["Amount"], 2) / 100.0

    # Convert binary Currency back to string
    transaction_data["Currency"] = ''.join(chr(int(binary_data["Currency"][i:i+8], 2)) for i in range(0, len(binary_data["Currency"]), 8))

    # Convert binary Card Type back to string
    transaction_data["Card Type"] = ''.join(chr(int(binary_data["Card Type"][i:i+8], 2)) for i in range(0, len(binary_data["Card Type"]), 8))

    # Convert binary Cardholder Name back to string
    transaction_data["Cardholder Name"] = ''.join(chr(int(binary_data["Cardholder Name"][i:i+8], 2)) for i in range(0, len(binary_data["Cardholder Name"]), 8))

    return transaction_data



In [5]:
# For every credit card transaction.
for transaction in transactions:
    
    # Convert into binary format, then seperate into strings of 2-binary digits.
    binary = encode_binary(transaction)
    binary_dict = {}
    
    for (key, value) in binary.items():
        # This array holds the binary string in two-bit segments
        binary_message = []
        
        for i in range(0, len(value),2):
            # For each two digits, prepare it to be sent into superdense
            binary_message.append(value[i:i+2])
        
        # Store the result with the corresponding attribute (time, merchant, amount etc.)
        binary_dict[key] = superdense_algorithm(binary_message, True)

    print(decode_binary(binary_dict))

Sending 00 to Bob.
Alice has transferred her qubit to Bob!
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
Bob Measures: 00

Sending 01 to Bob.
Applied Gate X to Alice's Qubit.
Alice has transferred her qubit to Bob!
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
Bob Measures: 01

Sending 00 to Bob.
Alice has transferred her qubit to Bob!
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
Bob Measures: 00

Sending 11 to Bob.
Applied Gate Z to Alice's Qubit.
Applied Gate X to Alice's Qubit.
Alice has transferred her qubit to Bob!
[ 0.+0.j  0.+0.j  0.+0.j -1.+0.j]
Bob Measures: 11

Sending 10 to Bob.
Applied Gate Z to Alice's Qubit.
Alice has transferred her qubit to Bob!
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
Bob Measures: 10

Sending 00 to Bob.
Alice has transferred her qubit to Bob!
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
Bob Measures: 00

Sending 01 to Bob.
Applied Gate X to Alice's Qubit.
Alice has transferred her qubit to Bob!
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
Bob Measures: 01

Sending 10 to Bob.
Applied Gate Z to Alice's Qubit.
Alice has transferred her qubi